In [ ]:
##########################################################################
# Author: Christopher Thomas Goodwin
# Creation Date: 2024.04.09
# Summary: Uses sklearn to explore the data of the NSHWE Stimmungs- und Lageberichte files
##########################################################################